In [73]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input, Concatenate
from keras.optimizers import RMSprop
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [53]:
root_path = "<kaggle dir here>"
img_path = root_path + "images/"
mask_path = root_path + "masks/"

In [71]:
train_fns = os.listdir(img_path)

In [74]:
train_imgs = [np.array(cv2.imread(img_path + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_fns)]
train_imgs = np.array(train_imgs)/255
train_imgs = np.expand_dims(train_imgs, axis=3)

100%|██████████| 4000/4000 [00:03<00:00, 1195.43it/s]


In [75]:
train_masks = [np.array(cv2.imread(mask_path + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_fns)]
train_masks = np.array(train_masks)/255
train_masks = np.expand_dims(train_masks, axis=3)

100%|██████████| 4000/4000 [00:01<00:00, 3071.84it/s]


In [76]:
X_train, X_val, y_train, y_val = train_test_split(
    train_imgs,
    train_masks,
    test_size=0.2)

In [77]:
def conv_block(num_layers, input_layer, units, kernel_size):
    x = input_layer
    for i in range(num_layers):
        x = Conv2D(units, kernel_size=kernel_size, padding='SAME',activation='relu')(x)
    return x

In [78]:
def build_model( input_layer ):
    cnn1 = conv_block(4, input_layer, 32, 3)
    cnn2 = conv_block(4, input_layer, 24, 5)
    cnn3 = conv_block(4, input_layer, 16, 7)
    concat_layers = Concatenate()([cnn1,cnn2,cnn3])
    dl_layer = Conv2D(16, 1, activation='relu')(concat_layers)
    output_layer = Conv2D(1,1, activation='sigmoid')(dl_layer)
    return output_layer

In [79]:
input_layer = Input( (101, 101, 1) )
output_layer = build_model( input_layer )
model = Model(input_layer, output_layer)
model.compile( loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'] )

In [80]:
model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs = 3, batch_size = 32)

Train on 3200 samples, validate on 800 samples
Epoch 1/3
3200/3200 [==============================] - 1185s 370ms/step - loss: 0.5459 - acc: 0.7530 - val_loss: 0.5125 - val_acc: 0.7480
Epoch 2/3
3200/3200 [==============================] - 1038s 324ms/step - loss: 0.4889 - acc: 0.7547 - val_loss: 0.4682 - val_acc: 0.7480
Epoch 3/3
3200/3200 [==============================] - 1094s 342ms/step - loss: 0.4681 - acc: 0.7782 - val_loss: 0.4698 - val_acc: 0.8047


In [81]:
model.save('salt_model')